# External-to-External Demand Inference

### Goal of the notebook
This notebook infers the external-to-external demand from the Streetlight Data SR262 link analysis. Please read the readme.md for the manual preprocessing before running this notebook. After the preprocessing, we can get the demand between some pairs of external centroids in the entire afternoon. Our goal is to estimate the demand for every 15 minutes based on the distribution of the flow measured by the flow detectors. 

***
**Outputs:** 

OD matrix:
- ext_ext_OD.csv

**Inputs:** 

Streetlight Raw Data:
- SR_262_Streetlight.pdf, available [here](https://www.dropbox.com/s/zg5mzys0nb2w7k1/SR_262_Streetlight.pdf?dl=0)

Preprocessed external-to-external demand in the afternoon:
- ext_ext_OD_AM_PM.csv, available [here](https://www.dropbox.com/s/i8dg5ba9pdvpfu7/ext_ext_OD_AM_PM.csv?dl=0) in /Manual-made dataset (do not touch)/Demand/OD demand/External to external OD demand StreetLight data 


Processed flow (both City and PeMS):
- Flow_processed_all.csv, available [here](https://www.dropbox.com/s/euphnmhaoxmtc39/Flow_processed_all.csv?dl=0) in /Processed data (do not touch)/Demand/Flow_speed

**Dependent scripts:** 
None

**Useful Information**
- Email from the City of Fremont explaining the input file, available [here](https://www.dropbox.com/s/m6bwbxpu3czrrh4/email.txt?dl=0) 

***
**Work done by the code:** 
- Estimating the distribution of demand based on flow measurements

## Global variable

In [129]:
# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fremontdropbox import get_dropbox_location

path_dropbox = get_dropbox_location()
data_folder = os.path.join(path_dropbox, 'Private Structured data collection')

cols = ['Day 1 - 14:0']

## Load input files

In [ ]:
import pandas as pd
ext_ext_OD_AM_PM = pd.read_csv(os.path.join(data_folder,'Manual-made dataset (do not touch)/Demand/OD demand/External to external OD demand StreetLight data/ext_ext_OD_AM_Pm.csv'))

display(ext_ext_OD_AM_PM)

# need to check whether flow_all should be placed in this folder
flow_all = pd.read_csv(os.path.join(data_folder, 'Processed data (do not touch)\Demand\Flow_speed\Flow_processed_all.csv'))


## Estimate the distribution of demand based on flow measurements

In [131]:
# the id of pems detector to estimate the distribution
pems_nb_id = 278
pems_sb_id = 335

# extract flow for the pems detector and compute the distribution
flow_nb = flow_all[flow_all['Id']==pems_nb_id].loc[:, 'Day 1 - 14:0': 'Day 1 - 20:0']
flow_sb = flow_all[flow_all['Id']==pems_sb_id].loc[:, 'Day 1 - 14:0': 'Day 1 - 20:0']
flow_nb_percent = flow_nb / int(flow_nb.sum(axis=1))
flow_sb_percent = flow_sb / int(flow_sb.sum(axis=1))

In [132]:
# create new columns
for col in flow_nb.columns.tolist():
    ext_ext_OD_AM_PM[col]=0

# demand that has external centriod 13 as their origin
start_with_13 = (ext_ext_OD_AM_PM['Centroid_O']==13)
# demand that has external centriod 13 as their destination
end_with_13 = (ext_ext_OD_AM_PM['Centroid_D']==13)

In [133]:
for col in flow_nb.columns.tolist():
    ext_ext_OD_AM_PM.loc[start_with_13, col]=float(flow_sb_percent[col])*ext_ext_OD_AM_PM[start_with_13]['PM']
    ext_ext_OD_AM_PM.loc[end_with_13, col]=float(flow_nb_percent[col])*ext_ext_OD_AM_PM[end_with_13]['PM']

In [134]:
ext_ext_OD_AM_PM

,Centroid_O,Centroid_D,AM,PM,Day 1 - 14:0,Day 1 - 14:15,Day 1 - 14:30,Day 1 - 14:45,Day 1 - 15:0,Day 1 - 15:15,...,Day 1 - 17:45,Day 1 - 18:0,Day 1 - 18:15,Day 1 - 18:30,Day 1 - 18:45,Day 1 - 19:0,Day 1 - 19:15,Day 1 - 19:30,Day 1 - 19:45,Day 1 - 20:0
0,13,4,810,720,29.514188,29.514188,29.131339,30.941171,29.688210,28.957316,...,33.655919,30.349495,26.590612,23.527819,25.546478,23.493015,21.265529,19.420892,17.785082,16.079664
1,13,20,13370,9140,374.666215,374.666215,369.806158,392.780974,376.875332,367.597042,...,427.243196,385.269976,337.553053,298.672596,324.298352,298.230773,269.954077,246.537439,225.771741,204.122396
2,13,23,10,10,0.409919,0.409919,0.404602,0.429738,0.412336,0.402185,...,0.467443,0.421521,0.369314,0.326775,0.354812,0.326292,0.295355,0.269735,0.247015,0.223329
3,13,21,3360,4850,198.810847,198.810847,196.231933,208.423164,199.983081,195.059699,...,226.710011,204.437569,179.117320,158.486006,172.083917,158.251559,143.246967,130.821289,119.802291,108.314400
4,4,13,390,380,18.961686,16.542644,12.846467,12.170337,10.697877,10.427425,...,18.871535,17.444150,17.369025,17.894903,19.833142,19.277213,16.031790,15.731288,14.409078,15.280535
5,20,13,6760,10980,547.892926,477.995334,371.195287,351.658693,309.112332,301.297695,...,545.288047,504.044126,501.873394,517.068522,573.073425,557.010004,463.234352,454.551421,416.346526,441.527025
6,23,13,0,10,0.498992,0.435333,0.338065,0.320272,0.281523,0.274406,...,0.496619,0.459057,0.457080,0.470919,0.521925,0.507295,0.421889,0.413981,0.379186,0.402119
7,21,13,3600,4480,223.548298,195.029062,151.453086,143.481871,126.122336,122.933850,...,222.485469,205.657348,204.771658,210.971492,233.822308,227.268198,189.006366,185.463604,169.875450,180.149460
8,13,22,30,30,1.229758,1.229758,1.213806,1.289215,1.237009,1.206555,...,1.402330,1.264562,1.107942,0.980326,1.064437,0.978876,0.886064,0.809204,0.741045,0.669986
9,22,13,10,710,35.428413,30.908624,24.002610,22.739314,19.988138,19.482820,...,35.259974,32.593017,32.452651,33.435214,37.056660,36.017951,29.954134,29.392669,26.922225,28.550473


## Save the output csv

In [135]:
save_path = os.path.join(data_folder, 'Manual-made dataset (do not touch)\Demand\OD demand\External to external OD demand StreetLight data\ext_ext_OD.csv')
ext_ext_OD_AM_PM.to_csv(save_path)

In [136]:
# correspondence between external centroid id and flow id
# if the origin is 13, the travel is going southbound. 
# Otherwise, the travel is going northbound
# flow_id_ext_int = {4: 121, 23:160, 22:163, 21:83, 20:245, 13:350}
# flow_id_int_ext = {4: 122, 23:159, 22:164, 21:84, 20:302, 13:299}